In [1]:
import os
%conda install -Uqq fastai
%conda install -Uqq timm
    
import timm
from fastai.vision.all import *
from fastdownload import download_url
from fastcore.all import *
from time import sleep

model = 'convnext_tiny_in22k'

/Users/memetral/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/memetral/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <248833C6-5096-36BC-A292-7EFB494EA582> /Users/memetral/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <1E9FA061-EA31-3736-81D0-79A33B965097> /Users/memetral/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Introduction

Every few years I take some portion of the wonderful fast.ai Practical Deep Learning for Coders course. Then I sit down to try and solve a very simple problem, get stuck, try and get help and have others shrug their shoulders and then eventually give up. This is just the next round of that. Sooner or later the systems will be good enough, is it now!?!

The goal is to classify cards from the game of SET. Each card has a color, a shape, a number of that shape and a fill pattern of the shape. So we need a classification system. Traditional CV would work fine, but is a pain to get right in terms of lighting and all that stuff. This SHOULD be a piece of cake for deep learning.

In [2]:
colorAttr = using_attr(RegexLabeller(r'_(red|green|purple)-'), 'name')
shapeAttr = using_attr(RegexLabeller(r'-(diamond|oval|squiggle)-'), 'name')
fillAttr = using_attr(RegexLabeller(r'-(empty|filled|hashed)-'), 'name')
countAttr = using_attr(RegexLabeller(r'-(single|double|triple).'), 'name')
exPath = Path("board2_red-diamond-hashed-single.png")
test_eq(colorAttr(exPath), 'red')
test_eq(shapeAttr(exPath), 'diamond')
test_eq(fillAttr(exPath), 'hashed')
test_eq(countAttr(exPath), 'single')

In [3]:
colorBlock = DataBlock( 
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=colorAttr,
    item_tfms=[Resize(192, method='squish')],
    batch_tfms=[DihedralItem,
                RandomResizedCrop(64,ratio=(.25,2)),
                Flip, 
                Rotate,
                Zoom(.8,1.2),
                Warp,
                Brightness,
                Contrast,
                Saturation]
).dataloaders('./images/train/', bs=16)
# colorBlock.show_batch(max_n=60)

In [4]:
colorLearner = vision_learner(colorBlock, model, metrics=error_rate)
colorLearner.fine_tune(20)